In [1]:
import seagall as sgl
import pandas as pd
import scanpy as sc

# Peaks

In [3]:
adata=sc.read_h5ad("RawData/MouseBrain_GEX.h5ad")
adata

AnnData object with n_obs × n_vars = 2488 × 2237
    obs: 'CellType', 'n_features', 'log_n_features', 'n_counts', 'log_n_counts', 'n_genes_by_counts', 'total_counts', 'total_counts_MT', 'pct_counts_MT', 'doublet_score', 'predicted_doublet', 'passes_filter', 'target'
    var: 'gene_ids', 'feature_types', 'genome', 'interval', 'n_cells', 'log_n_cells', 'MT', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'passes_filter', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'hvg', 'inv_map', 'log1p', 'map', 'scrublet'

In [4]:
sgl.ee.geometrical_graph(adata, target_label="CellType", path="SEAGALL")

Fitting GRAE
       Fitting GRAE...
           Fitting manifold learning embedding...
           Fitting encoder & decoder...


In [5]:
adata

AnnData object with n_obs × n_vars = 2488 × 2237
    obs: 'CellType', 'n_features', 'log_n_features', 'n_counts', 'log_n_counts', 'n_genes_by_counts', 'total_counts', 'total_counts_MT', 'pct_counts_MT', 'doublet_score', 'predicted_doublet', 'passes_filter', 'target'
    var: 'gene_ids', 'feature_types', 'genome', 'interval', 'n_cells', 'log_n_cells', 'MT', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'passes_filter', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'hvg', 'inv_map', 'log1p', 'map', 'scrublet'
    obsm: 'GRAE_latent_space'
    layers: 'GRAE_decoded_matrix'
    obsp: 'GRAE_graph'

In [ ]:
adata.write("MouseBrain_GEX_Step1.h5ad", compression="gzip")

In [ ]:
sgl.ee.classify_and_explain(adata, target_label="CellType", path="SEAGALL", hypopt=0.25)

In [ ]:
adata.write("Test_Def.h5ad", compression="gzip")

In [ ]:
for ct in sorted(list(adata.obs.CellType)):
    plt.scatter(x=range(0, len(adata.var)), y=sorted(adata.var[f"SEAGALL_Importance_for_{ct}"])), c=

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
adata=sc.read_h5ad("Test_Def.h5ad")
label="CellType"
adata

In [ ]:
mydata = sgl.mlu.create_pyg_dataset(adata, label)
mydata = torch_geometric.transforms.RandomNodeSplit(num_val=0.15, num_test=0.15)(mydata)

In [ ]:
model = sgl.mlu.GAT(n_feats=mydata.num_features, n_classes=mydata.num_classes).to(device)
optimizer_model = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-3)
class_weights = sklearn.utils.class_weight.compute_class_weight(class_weight='balanced',classes=np.unique(mydata.y), y=mydata.y.numpy())
criterion = torch.nn.CrossEntropyLoss(weight=torch.tensor(class_weights, dtype=torch.float), reduction="mean")
model, history = sgl.mlu.GAT_train_node_classifier(model, mydata, optimizer_model, criterion, "TEMP.pth", epochs=30, patience=5)

In [ ]:
adata.uns["GAT_history"]=history

In [ ]:
model.eval()
pred = model(mydata.x, mydata.edge_index).argmax(dim=1)

adata.obs["SEAGALL_set"] = "--"
adata.obs["SEAGALL_prediction"] = [adata.uns["inv_map"][str(num)] for num in list(pred.cpu().detach().numpy())]
adata.obs.loc[mydata.train_mask.cpu().detach().numpy(),"SEAGALL_set"] = "Train"
adata.obs.loc[mydata.val_mask.cpu().detach().numpy(),"SEAGALL_set"] = "Validation"
adata.obs.loc[mydata.test_mask.cpu().detach().numpy(),"SEAGALL_set"] = "Test"

In [ ]:
explainer = torch_geometric.explain.Explainer(
            model=model,
            algorithm=torch_geometric.explain.GNNExplainer(epochs=30),
            explanation_type='model',
            node_mask_type='attributes',
            edge_mask_type='object',
            model_config=dict(mode='multiclass_classification', task_level='node', return_type='probs'))

In [ ]:
adata.layers["Importance"]=scipy.sparse.csr_matrix(explainer(x=mydata.x, edge_index=mydata.edge_index).node_mask.numpy(), dtype="float32")

In [ ]:
for gt in sorted(set(adata.obs[label])):
    imps = np.array(adata[adata.obs[label]==gt].layers["Importance"].mean(axis=0)).reshape(adata.shape[1], )
    adata.var[f"Importance_for_{gt}"] = imps

In [ ]:
n_feat=50
gts = sorted(set(adata.obs[label]))
jc = pd.DataFrame(index=gts, columns=gts)
for gti in gts:
    for gtj in gts[gts.index(gti)+1:]:
        fsi = adata[adata.obs[label]==gti].var[f"Importance_for_{gti}"].sort_values()[::-1][:int(n_feat)].index
        fsj = adata[adata.obs[label]==gtj].var[f"Importance_for_{gtj}"].sort_values()[::-1][:int(n_feat)].index
        jc.at[gti, gtj] = len(sgl.ut.intersection([fsi, fsj]))/len(sgl.ut.flat_list([fsi, fsj]))

In [ ]:
adata.uns[f"SEAGALL_Top_{n_feat}_Specificty"]=jc.values.astype()

In [ ]:
adata

In [ ]:
adata.write("Test_Def_All.h5ad", compression="gzip")

In [ ]:
pd.DataFrame.from_dict(adata.uns["GAT_history"]).melt()